In [1]:
# 这个文件用来检查model1的信号转好功能是否正常
# 这个功能很容易出错

import os
import sys
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

parent_path = os.path.dirname(os.getcwd())
tools_path = os.path.join(parent_path, 'tools')
sys.path.append(parent_path)
sys.path.append(tools_path)

from tools.path import get_env_path, get_prompt_from_file
from tools.CTA_GEP import GeneticProgram
from tools.backtest import BacktestEngine
from tools.more_operators import *
from tools.evaluate_and_visualizations import *

In [2]:
from dotenv import load_dotenv
env = get_env_path('project')
load_dotenv(env)

True

In [3]:
from langchain_core.messages import (
    BaseMessage,
    ToolMessage,
    HumanMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from pandas import DataFrame


In [4]:
#GEP_str = "add(\n\tKAMA(neg(_open), _close),\n\tBIAS\n)"
#GEP_str = "add(\n\tMA(BIAS, _open),\n\tBIAS\n)"
#GEP_str = "add(\n\tBIAS,\n\tts_corr(ts_mean_return(delta(ts_stddev(EMA, SMA), SMA), DX), ts_mean_return(_low, MIDPRICE(sin(8), EMA, RSI)), ATR)\n)"
GEP_str = "add(\n\tmul(BIAS, protected_div(ATR, MA(BIAS, DX))),\n\tts_corr(ts_argmin(_low, EMA), _volume, MA(_high, RSI))\n)"

In [5]:
llm = ChatOpenAI(model="gpt-4-1106-preview")
system_prompt = get_prompt_from_file("model_1_backtest_node")

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

# 打开和读取文件
with open('/Users/syesw/Desktop/Generative_AI_Quant/prompts/supported_operators.txt', 'r') as file:
    supported_operators = file.read()

with open('/Users/syesw/Desktop/Generative_AI_Quant/prompts/supported_factors.txt', 'r') as file:
    supported_factors = file.read()

prompt = prompt.partial(GEP_str=GEP_str)
prompt = prompt.partial(supported_operators=supported_operators)
prompt = prompt.partial(supported_factors=supported_factors)
signal_chain = prompt | llm
result = signal_chain.invoke(['help me convert the str to a signal equation'])

signal_equation = str(result.content)

In [6]:
signal_equation

"signals = add(\n    mul(self.df.loc[i, 'BIAS'], protected_div(self.df.loc[i, 'ATR'], MA(self.df.loc[i, 'BIAS'], self.df.loc[i, 'DX']))),\n    ts_corr(ts_argmin(self.df.loc[i, 'low'], self.df.loc[i, 'EMA']), self.df.loc[i, 'volume'], MA(self.df.loc[i, 'high'], self.df.loc[i, 'RSI']))\n)"